# Lab 6: Logistic Regression
In this lab, we will study logistic regression and we will build a logistic regression model to predict whether a student gets admitted into the college.

More specifically, the dataset that we will employ contains the scores of $80$ students on two standardized exams. Half of the students were admitted to college, while the other half of the students were not admitted. The dataset can be downloaded from <a href="http://openclassroom.stanford.edu/MainFolder/courses/MachineLearning/exercises/ex4materials/ex4Data.zip">here</a>.

Suppose that you are the administrator of a college and you want to determine each applicant’s chance of admission based on their results on these two exams. Your task is to build a binary classification model that estimates college admission chances based on a student's scores on these two exams. To this end, you will employ a logistic regression classifier.

You are given the following function that reads the data from the disk. 

In [1]:
import numpy as np

def read_data():
    X = np.loadtxt('ex4Data/ex4x.dat')
    y = np.loadtxt('ex4Data/ex4y.dat')
    
    return X, y

Use the above function to load the data and the class labels. The function returns a matrix $\mathbf{X}$ and a vector $\mathbf{y}$. The first column of matrix $\mathbf{X}$ represents all Exam $1$ scores, and the second column represents all Exam $2$ scores. As regards vector $\mathbf{y}$, it uses $1$ to label a student who was admitted and $0$ to label a student who was not admitted.

Compute and print the minimum, maximum, median and mean scores on the two exams. For this task, you can use the built-in functions `min, max, median, mean` of the `NumPy` library.

In [1]:
#your code here

Your next task is to plot the data in a $2$-dimensional plane using `scatter` (http://matplotlib.org/api/pyplot_api.html#matplotlib.pyplot.scatter). Use different colors to represent the two classes.

In [2]:
#your code here

We will next train a logistic regression model on the dataset. Make use of the `LogisticRegression` object provided by `scikit-learn` (http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html). Set the value of parameter $C$ equal to $10$. After training the model, use it to predict whether the students of the dataset were admitted to college or not. Measure the performance of the logistic regression classifier by computing its accuracy. To do this, you can use the accuracy_score function of scikit-learn (http://scikit-learn.org/stable/modules/generated/sklearn.metrics.accuracy_score.html). 

In [3]:
#your code here

Does the classifier yield high performance on the dataset it was trained?

---

The following code plots the decision boundary of the logistic regression classifier. The datapoints are colored according to their labels. Run the code to visualize the boundary.

In [4]:
# Plot the decision boundary
x_min, x_max = X[:, 0].min() - 5, X[:, 0].max() + 5
y_min, y_max = X[:, 1].min() - 5, X[:, 1].max() + 5

plot_x = np.array([x_min, x_max])
plot_y = (- 1.0 / logreg.coef_[0,1]) * (logreg.coef_[0,0] * plot_x + logreg.intercept_)
plt.plot(plot_x, plot_y)
plt.scatter(X[:, 0], X[:, 1], c=y, cmap=custom_cmap)
plt.xlabel('Exam 1 score')
plt.ylabel('Exam 2 score')

plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)

What do you observe?

---

You will next write your own algorithm for performing logistic regression. 

In contrast to regression problems, classification problems need binary answers $\rightarrow \hat{y} \in \{0, 1\}$. Therefore, we cannot use least squares as a measure of error, as it may penalize correct answers. For example, if the output of linear regression is $f(\mathbf{x}) = 5.4$ and the true class is $1$, it's not a good practice to use it for a decision where: 
$$f(\mathbf{x}) \ge 0.5 \Rightarrow \hat{y} \in 1$$
$$f(\mathbf{x}) < 0.5 \Rightarrow \hat{y} \in 0$$
since squared error penalizes it although correct.

It is thus necessary to fix functions $f(\mathbf{x})$ to give binary output. To output binary, we use the `sigmoid` function (https://en.wikipedia.org/wiki/Sigmoid_function), also known as the logistic or logit function, and defined as follows:

$$ g(z) = \frac{1}{1 + e^{-z}} $$

The output of the logistic regression has a probabilistic interpretation:

$$ g(\mathbf{x_i}^T\boldsymbol{\theta}) = p(\hat{y}_i=1|\mathbf{x_i},\boldsymbol{\theta}) \rightarrow \text{ Probability of class 1} $$

Your next task is to implement the sigmoid function. Specifically, write a function that takes as input a real number and returns the value of the sigmoid function for that number. When you are finished, test a few values. For large positive values, the function should return values close to $1$, while for large negative values, it should return values close to $0$.

In [5]:
#your code here

The following code plots the sigmoid function. Run the code and comment on the plot.

In [6]:
x = np.arange(-8., 8., 0.2)
w = np.zeros(x.shape)

for i in range(x.shape[0]):
    w[i] = sigmoid(x[i])
    
plt.plot(x, w)

A Bernoulli random variable $x$ takes values in $\{0, 1\}$ (https://en.wikipedia.org/wiki/Bernoulli_distribution):

$$ p(x|\mu) = \left\{
\begin{array}{lr}
\mu & \text{if }x = 1,\\
1-\mu & \text{if }x = 0,
\end{array}
\right.
$$

where $\mu \in [0, 1]$. We can write this probability more succinctly as follows:

$$ p(x|\mu) = \mu^x (1-\mu)^{1-x} $$

The logistic regression model specifies the probability of a binary output $\hat{y}_i \in \{0, 1\}$ given the input $\mathbf{x}_i$ as follows:

$$ p(\mathbf{y}|\mathbf{X}, \boldsymbol{\theta}) = \prod_{i=1}^n Ber(y_i|g(\mathbf{x}_i^T \boldsymbol{\theta})) = \prod_{i=1}^n \Big[ \frac{1}{1+e^{-\mathbf{x}_i^T \boldsymbol{\theta}}} \Big]^{y_i} \Big[1 - \frac{1}{1+e^{-\mathbf{x}_i^T \boldsymbol{\theta}}} \Big]^{1-y_i} = \prod_{i=1}^n \pi_i^{y_i} (1-\pi_i)^{1-y_i} $$

where $\pi_i = p(y_i=1|\mathbf{x_i},\boldsymbol{\theta})$ and $1-\pi_i = p(y_i=0|\mathbf{x_i},\boldsymbol{\theta})$. The loglikelihood is given by:

$$ L(\boldsymbol{\theta}) = \log p(\mathbf{y}|\mathbf{X}, \boldsymbol{\theta}) = \sum_{i=1}^n \Big( y_i \log \pi_i - (1-y_i)\log(1-\pi_i) \Big) $$

Likelihood measures the goodness of fit. Error is the opposite of loglikelihood and is thus equal to:

$$ J(\boldsymbol{\theta}) = -\log p(\mathbf{y}|\mathbf{X}, \boldsymbol{\theta}) = -\sum_{i=1}^n \Big( y_i \log \pi_i + (1-y_i)\log(1-\pi_i) \Big) $$

Write a function that takes as input a data matrix $\mathbf{X}$, the class labels $\mathbf{y}$ and some parameters $\boldsymbol{\theta}$ and returns the error $J(\boldsymbol{\theta})$ (i.e. negative loglikelihood). Use the equation given above. The structure of your code should be as follows:
```python
def compute_error(theta, X, y):
    
    J = 0 # initialize error
    
    # Body of for function

    return J
```

In [7]:
#your code here

Logistic regression can be expressed as the following minimization problem:

$$ \min_{\boldsymbol{\theta}} J(\boldsymbol{\theta}) $$

Our goal is thus to compute the parameters $\boldsymbol{\theta}$ that maximize the likelihood (i.e. minimize the error). The error function $J(\boldsymbol{\theta})$ is convex. However, even if it is convex, we cannot obtain the minimum solution in closed form. An alternative approach is to use gradient descent to minimize $J(\boldsymbol{\theta})$ . Since $J(\boldsymbol{\theta})$ is convex, there is a unique minimum solution, and gradient descent always converges to that solution.
  
As we have mentioned in a previous lab, `gradient descent` is an optimization method that starts with some “initial guess” for $\boldsymbol{\theta}$, and repeatedly changes $\boldsymbol{\theta}$ to make $J(\boldsymbol{\theta})$ smaller. More specifically, gradient descent repeatedly performs the update:

$$\theta_j = \theta_j - \alpha \frac{\partial J(\boldsymbol{\theta})}{\partial \theta_j}$$

where $\alpha$ is the learning rate. Hence, we first have to compute the derivative $\frac{\partial J(\boldsymbol{\theta})}{\partial \theta_j}$. To do this, we will need the following:
1. The derivative of a logistic function is:
	$$ \frac{\partial g(z)}{\partial z} = g(z)(1-g(z)) $$
2. The derivative $\frac{\partial \mathbf{x}^T \boldsymbol{\theta}}{\partial \theta_j}$ is equal to:
	$$ \frac{\partial \mathbf{x}^T \boldsymbol{\theta}}{\partial \theta_j} = \frac{\partial}{\partial \theta_j} (\theta_0x_0 + \ldots + \theta_jx_j + \ldots + \theta_nx_n) = x_j $$
3. The derivative of the natural logarithm is:
	$$ \frac{\partial \log x}{\partial x} = \frac{1}{x} $$
  

For the case of one training example $D = \{\mathbf{x},y\}$, the partial derivative is (using the equations above):

$$ \frac{\partial J(\boldsymbol{\theta})}{\partial \theta_j} = -\frac{\partial}{\partial \mathbf{x}^T \boldsymbol{\theta}} \Big(  y \log\pi + (1-y)\log(1-\pi) \Big)\frac{\partial \mathbf{x}^T \boldsymbol{\theta}}{\partial \theta_j} = -\frac{\partial}{\partial \mathbf{x}^T \boldsymbol{\theta}} \Big(  y \log\pi_i + (1-y)\log(1-\pi) \Big) x_j $$

Furthermore,

$$ \frac{\partial}{\partial \mathbf{x}^T \boldsymbol{\theta}} \Big( y\log\pi+(1-y)\log(1-\pi)\Big) = y\frac{1}{\pi}\frac{\partial \pi}{\partial \mathbf{x}^T \boldsymbol{\theta}} + (1-y)\frac{-1}{1-\pi}\frac{\partial \pi}{\partial \mathbf{x}^T \boldsymbol{\theta}} = y(1-g(\mathbf{x}^T \boldsymbol{\theta})) + (1-y)(-g(\mathbf{x}^T \boldsymbol{\theta})) = y-g(\mathbf{x}^T \boldsymbol{\theta}) $$
  
Hence, the partial derivative is given by:

$$ \frac{\partial J(\boldsymbol{\theta})}{\partial \theta_j} = -\big(y-g(\mathbf{x}^T \boldsymbol{\theta})\big) x_j $$

Therefore, for a single training example, this gives the update rule:

$$ \theta_j = \theta_j + \alpha (y - g(\mathbf{x}^T \boldsymbol{\theta})) x_j $$

and for all training examples:

$$ \theta_j = \theta_j + \alpha \sum_{i=1}^n(y - g(\mathbf{x}^T \boldsymbol{\theta})) x_{ij} $$

Write a function that takes as input a training matrix $\mathbf{X}$, its true outputs $\mathbf{y}$ and the initial parameters $\boldsymbol{\theta}$, and returns the gradients of the error function with respect to the parameters  $\boldsymbol{\theta}$. Use the equation given above. Compute the gradients by looking at every example in the entire training set. Below you can see the basic points and structure of the code for this task:
```python
def compute_grads(theta, X, y):
    
    grads = np.zeros(np.size(theta)) # initialize gradient
    
    # Body of for function

    return grads
```

In [8]:
#your code here

As in the case of linear regression, for each training example $i$, we assume that $x_{i0} = 1$. This corresponds to the intercept of the linear model. Therefore, your next task is to add an extra column to matrix $\mathbf{X}$ whose entries are all equal to $1$ to accommodate the intercept term. This should correspond to the first column of $\mathbf{X}$. Furthermore, initialize parameters $\boldsymbol{\theta}$ to random values sampled from a Gaussian distribution with zero mean and standard deviation equal to $0.1$.

In [9]:
#your code here

Run the follwing code to perform gradient descent and to minimize the error function $J(\boldsymbol{\theta})$.

In [10]:
import scipy.optimize as op

Result = op.minimize(fun = compute_error, x0 = initial_theta, args = (X, y), method = 'TNC',jac = compute_grads);
theta = Result.x;

Use the following code to plot the decision boundary of the logistic regression classifier. Compare it to the boundary generated by the logistic regression classifier of `scikit-learn`.

In [11]:
# Plot the decision boundary
x_min, x_max = X[:, 1].min() - 5, X[:, 1].max() + 5
y_min, y_max = X[:, 2].min() - 5, X[:, 2].max() + 5

plot_x = np.array([x_min, x_max])
plot_y = (- 1.0 / theta[2]) * (theta[1] * plot_x + theta[0])
plt.plot(plot_x, plot_y)
plt.scatter(X[:, 1], X[:, 2], c=y, cmap=custom_cmap)
plt.xlabel('Exam 1 score')
plt.ylabel('Exam 2 score')

plt.xlim(x_min, x_max)
plt.ylim(y_min, y_max)

What do you observe?

---

Write another function that takes as input a data point $\mathbf{x}$ and the learned parameters $\boldsymbol{\theta}$, and returns the predicted class for $\textbf{x}$. Use the sigmoid function defined above.

In [12]:
#your code here

Use the function you created above to predict if the students of the dataset were admitted to college or not. Compute the accuracy of your logistic regression classifier and compare its performance to the performance of the classifier provided by `scikit-learn`.

In [13]:
#your code here

Are the two accuracies equal to each other?

---

Given a new student who scored $60$ on the first exam and $50$ on the second, predict if he will be admitted to college or not.

In [14]:
#your code here